# #5 Crucipixel Game - Formulation
_Author: Éder Pinheiro_  
_Aug, 2021_

This is the MIP formulation of the puzzle. Statement and solution implementation of all puzzles 
are available from the main page of the [Fun Puzzles](https://mip-master.github.io/puzzles/) project, 
which is maintained by [Mip Master](https://mipmaster.org/).

The purpose game is to darken some of the cells so that in every row and every collumn the darkened cells form distinct strings with the lengths and in the correct order prescribed by the numbers on the left of the row and on top of the column.


### Input Data  
### Decision Variables
### Constraints
### Objective Function
### Final Formulation

## Outline - at work (formulation and implemantation)
<img src="6_crucipixel_game.png" alt="Drawing" style="width: 10cm;"/>


Based on https://run.unl.pt/bitstream/10362/2388/1/Mingote_2009.pdf

Parameters:

$n,m$ numbers of rows and columns, respectivaly.

$s^c_j$ quantity of strings in column $j$.

$s^r_i$ quantity of string in row $i$.

$l^{cj}_s$ string $s$ size in column $j$

$l^{ri}_s$ string $s$ size in row $i$

Variables:

$x_{ijs}$ if string s of column j starts in row i

$y_{ijs}$ if string s of row i starts in column j

$z_{ij}$ if cell with row $i$ an column $j$ is painted

$$ \text{Every strings in solution}$$
$$\sum_{i=1}^n x_{ijs} = 1; \quad  j = 1, \cdots m; \quad s = 1, \cdots, s^c_j$$
$$\sum_{j=1}^m y_{ijs} = 1; \quad  i = 1, \cdots n; \quad s = 1, \cdots, s^r_i$$

$$\text{Strings sequence in right order:}$$
$$x_{ijs} \leq \sum_{k=i+l^{cj}_s+1}^n x_{kj(s+1)}; \quad  j = 1, \cdots m; \quad s = 1, \cdots, (s^c_j-1);$$
$$y_{ijs} \leq \sum_{k=j+l^{ri}_s+1}^m y_{ik(s+1)}; \quad  i = 1, \cdots n; \quad s = 1, \cdots, (s^r_i-1);$$

$$\text{The cells where the strings are placed are daken:}$$
$$z_{ij} \geq  y_{iks}; \quad i = 1,\cdots, n; \quad s = 1, \cdots, s^r_i; \quad k = j-l^{ri}_s+1, \cdots, j$$
$$z_{ij} \geq  x_{kjs}; \quad j = 1,\cdots, m; \quad s = 1, \cdots, s^c_j; \quad k = i-l^{cj}_s+1, \cdots, i$$

In [3]:
import pulp
Sc = [[1], [2,1], [], []]#[[1,1,3], [1,1,1], [1,1,1], [1,2,1], [3,3], [3,2,3], [3,4], [1,1], [1,2,1,2], [6,2]]
Sr= [[1,1], [2], [1], []]#[[4,1,2], [3,1], [1,3,2], [1,1,1,2], [1,1,1,1], [1,1,1,1], [1,1,1,4], [1,3], [1,4,2], [2,2]]
m = len(Sc)
n = len(Sr)
keys_x=[(i,j,s) for j in range(m) for s in range(len(Sc[j])) for i in range(n)]
keys_y=[(i,j,s) for i in range(n) for s in range(len(Sr[i])) for j in range(m)]
x = pulp.LpVariable.dicts(indexs=keys_x, cat=pulp.LpBinary, name='x')
y = pulp.LpVariable.dicts(indexs=keys_y, cat=pulp.LpBinary, name='y')
z = pulp.LpVariable.dicts(indexs=[(i,j) for i in range(n) for j in range(m)], cat=pulp.LpBinary, name='z')
model = pulp.LpProblem('', sense=pulp.LpMinimize)

for j in range(m):
    for s in range(len(Sc[j])):
        model.addConstraint(pulp.lpSum(x[i, j, s] for i in range(n)) == 1, name=f'column_string_{s}{j}')
for i in range(n):
    for s in range(len(Sr[i])):
        model.addConstraint(pulp.lpSum(y[i, j, s] for j in range(m)) == 1, name=f'row_string_{s}{i}')
        
for j in range(m):
    for s in range(len(Sc[j])-1):
        model.addConstraint(x[i,j,s] <= pulp.lpSum(x[k, j, s] for k in range(i+Sc[j][s]+1,n+1)), name=f'string_{s}{j}_above_the_{s-1}{j}')      
for i in range(n):
    for s in range(len(Sr[i])-1):
        model.addConstraint(y[i,j,s] <= pulp.lpSum(y[i, k, s] for k in range(j+Sr[i][s]+1,m+1)), name=f'string_{s}{i}_left_from_{s-1}{i}')
        
for i in range(n):
    for j in range(m):
        for s in range(len(Sr[i])):
            for k in range(max(j-Sr[i][s]+1,0), j+1):
                model.addConstraint(z[i,j] >= y[i, k, s], name=f'z_{i}{j}_y_{i}{k}{s}')

for i in range(n):
    for j in range(m):
        for s in range(len(Sc[j])):
            for k in range(max(i-Sc[j][s]+1,0),i+1):
                model.addConstraint(z[i,j] >= pulp.lpSum(x[k, j, s]), name=f'z_{i}{j}_x_{k}{j}{s}')
            
# for i in range(n):
#     for j in range(m):
#         model.addConstraint(z[i,j] <= pulp.lpSum(y[i, k, s] for s in range(len(Sr[i])) for k in range(max(j-Sr[i][s]+1,0), m)), name=f'z_{i}{j}_sumy')
        
# for i in range(n):
#     for j in range(m):
#         model.addConstraint(z[i,j] <= pulp.lpSum(x[k, j, s] for s in range(len(Sc[j])) for k in range(max(i-Sc[j][s]+1,0),n)), name=f'z_{i}{j}_sumx')
        
# set the objective function
model.setObjective(pulp.lpSum(z[(i, j)] for i in range(n) for j in range(m)))


# region Optimize and retrieve the solution
model.solve()

# retrieve and print out the solution
z_sol = [[z[i,j].value() for j in range(m)] for i in range(n)]
# print(f'z = {z_sol}')
for row in z_sol:
    print(row)
# endregion

print(model.status)
for var_x in x.items():
    print(var_x, var_x[1].value())
print(model)

len(model.constraints)

[1.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 1.0, 0.0]
[0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
1
((0, 0, 0), x_(0,_0,_0)) 1.0
((1, 0, 0), x_(1,_0,_0)) 0.0
((2, 0, 0), x_(2,_0,_0)) 0.0
((3, 0, 0), x_(3,_0,_0)) 0.0
((0, 1, 0), x_(0,_1,_0)) 0.0
((1, 1, 0), x_(1,_1,_0)) 1.0
((2, 1, 0), x_(2,_1,_0)) 0.0
((3, 1, 0), x_(3,_1,_0)) 0.0
((0, 1, 1), x_(0,_1,_1)) 0.0
((1, 1, 1), x_(1,_1,_1)) 1.0
((2, 1, 1), x_(2,_1,_1)) 0.0
((3, 1, 1), x_(3,_1,_1)) 0.0
:
MINIMIZE
1*z_(0,_0) + 1*z_(0,_1) + 1*z_(0,_2) + 1*z_(0,_3) + 1*z_(1,_0) + 1*z_(1,_1) + 1*z_(1,_2) + 1*z_(1,_3) + 1*z_(2,_0) + 1*z_(2,_1) + 1*z_(2,_2) + 1*z_(2,_3) + 1*z_(3,_0) + 1*z_(3,_1) + 1*z_(3,_2) + 1*z_(3,_3) + 0
SUBJECT TO
column_string_00: x_(0,_0,_0) + x_(1,_0,_0) + x_(2,_0,_0) + x_(3,_0,_0) = 1

column_string_01: x_(0,_1,_0) + x_(1,_1,_0) + x_(2,_1,_0) + x_(3,_1,_0) = 1

column_string_11: x_(0,_1,_1) + x_(1,_1,_1) + x_(2,_1,_1) + x_(3,_1,_1) = 1

row_string_00: y_(0,_0,_0) + y_(0,_1,_0) + y_(0,_2,_0) + y_(0,_3,_0) = 1

row_string_10: y_(0,_0,_1) + 

43